In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.colors as pc
import plotly.graph_objects as go
import us # for location abbreviation

In [19]:
df = pd.read_csv("shopping_trends_updated.csv")
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3900 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

- donute chart: shipping type, payment method?, frequency of purchases
- sunburn chart: category, season
- map: location
- bar chart rating: rating 1-5
- bar chart age: 1-100
- bar chart purchase: age and purchase amount

In [21]:
cat = ['Gender', 'Item Purchased', 'Category',
       'Location', 'Size', 'Color', 'Season',
       'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used',
       'Payment Method']

for i in cat:                                                  # perform a loop to see the unique values of each categorical column type
    print(df[i].value_counts())
    print("\n")

Gender
Male      2652
Female    1248
Name: count, dtype: int64


Item Purchased
Blouse        171
Jewelry       171
Pants         171
Shirt         169
Dress         166
Sweater       164
Jacket        163
Belt          161
Sunglasses    161
Coat          161
Sandals       160
Socks         159
Skirt         158
Shorts        157
Scarf         157
Hat           154
Handbag       153
Hoodie        151
Shoes         150
T-shirt       147
Sneakers      145
Boots         144
Backpack      143
Gloves        140
Jeans         124
Name: count, dtype: int64


Category
Clothing       1737
Accessories    1240
Footwear        599
Outerwear       324
Name: count, dtype: int64


Location
Montana           96
California        95
Idaho             93
Illinois          92
Alabama           89
Minnesota         88
Nebraska          87
New York          87
Nevada            87
Maryland          86
Delaware          86
Vermont           85
Louisiana         84
North Dakota      83
Missouri          81
W

# Feature Engineering

In [22]:
df.drop(columns="Frequency of Purchases", axis=1, inplace=True) # delete the frequency of purchases column because we don't use it, unless we need it for segmentation

In [23]:
df["Difference Amount"] = df["Purchase Amount (USD)"] - df["Previous Purchases"]    # define difference amount column

In [24]:
df["State Code"] = df["Location"].map(                                      # Make a column from state/location abbreviation 
    lambda x: us.states.lookup(x).abbr if us.states.lookup(x) else None)

In [25]:
df.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Difference Amount', 'State Code'],
      dtype='object')

In [26]:
new_columns = ['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',  'Location', "State Code",
               'Size', 'Color', 'Season', 'Review Rating', 'Purchase Amount (USD)', 
               'Previous Purchases', 'Difference Amount', 'Subscription Status', 
               'Shipping Type', 'Discount Applied', 'Promo Code Used', 'Payment Method',
               ]

df = df.reindex(columns=new_columns)                                        # change the columns order
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Location,State Code,Size,Color,Season,Review Rating,Purchase Amount (USD),Previous Purchases,Difference Amount,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Payment Method
0,1,55,Male,Blouse,Clothing,Kentucky,KY,L,Gray,Winter,3.1,53,14,39,Yes,Express,Yes,Yes,Venmo
1,2,19,Male,Sweater,Clothing,Maine,ME,L,Maroon,Winter,3.1,64,2,62,Yes,Express,Yes,Yes,Cash
2,3,50,Male,Jeans,Clothing,Massachusetts,MA,S,Maroon,Spring,3.1,73,23,50,Yes,Free Shipping,Yes,Yes,Credit Card
3,4,21,Male,Sandals,Footwear,Rhode Island,RI,M,Maroon,Spring,3.5,90,49,41,Yes,Next Day Air,Yes,Yes,PayPal
4,5,45,Male,Blouse,Clothing,Oregon,OR,M,Turquoise,Spring,2.7,49,31,18,Yes,Free Shipping,Yes,Yes,PayPal


# Visualization

In [27]:
# grouping based on state location
df_state = df.groupby(["Location", "State Code"], as_index=False).agg({
    "Purchase Amount (USD)": "sum",
    "Age": "mean",
    "Customer ID": "count"
}).rename(columns={"Customer ID": "Purchase Count"})

# mapping based on state
fig = px.choropleth(
    data_frame=df_state,
    locations="State Code",
    color="Purchase Count",
    locationmode="USA-states",
    scope="usa",
    color_continuous_scale=pc.qualitative.Set3,
    hover_data={                                            # set the format to display
        "Location": True,
        "Purchase Amount (USD)": ":.0f",
        "Purchase Count": True,
        "Age": ":.1f"
    },
    labels={"Age": "Age (Avg)"}
).update_layout(                                            # modify the layout
        template='plotly_dark',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=0, b=0),
        height=200, width=500
    )
fig.show()

In [ ]:
# groupby by the item
df_item = df.groupby(["Season", "Item Purchased"], as_index=False).agg({
    "Purchase Amount (USD)": "sum",
    "Age": "mean",
    "Customer ID": "count"
}).rename(columns={"Customer ID": "Purchase Count"})

# groupby by the season
df_season = df.groupby("Season", as_index=False).agg({
    "Purchase Amount (USD)": "sum",
    "Age": "mean",
    "Customer ID": "count"
}).rename(columns={"Customer ID": "Purchase Count"})
df_season["Item Purchased"] = None 

df_season["id"] = "Season__" + df_season["Season"].astype(str)
df_season["parent"] = "All_Seasons"  

df_item["id"] = (
    "Season__" + df_item["Season"].astype(str) + "__Item__" + df_item["Item Purchased"].astype(str)
)
df_item["parent"] = "Season__" + df_item["Season"].astype(str)

root = pd.DataFrame({
    "Season": ["All"],
    "Item Purchased": [None],
    "Purchase Amount (USD)": [df["Purchase Amount (USD)"].sum()],
    "Age": [df["Age"].mean()],
    "Purchase Count": [df["Customer ID"].count()],
    "id": ["All_Seasons"],
    "parent": [""]
})

df_sunburst = pd.concat([root, df_season, df_item], ignore_index=True)

df_sunburst["label"] = df_sunburst["Item Purchased"].fillna(df_sunburst["Season"])

df_sunburst["id"] = df_sunburst["id"].astype(str)
df_sunburst["parent"] = df_sunburst["parent"].fillna("").astype(str)
df_sunburst.loc[df_sunburst["parent"] == "", "parent"] = ""  

df_sunburst["Season_str"] = df_sunburst["Season"].fillna("").astype(str)
df_sunburst["Item_str"] = df_sunburst["Item Purchased"].fillna("").astype(str)
custom = df_sunburst[["Season_str", "Item_str", "Purchase Amount (USD)", "Age"]].values

fig = px.sunburst(
    df_sunburst,
    ids="id",
    parents="parent",
    names="label",          
    values="Purchase Count",
    color="Season",         
    hover_data={
        "Season": True,
        "Item Purchased": True,
        "Purchase Amount (USD)": ":.0f",
        "Purchase Count": True,
        "Age": ":.1f"
    },
    branchvalues="total",
    color_discrete_sequence=pc.qualitative.Set3
)

fig.update_traces(
    customdata=custom,
    hovertemplate=(
        "<b>%{label}</b><br>"                              # display item/season label
        "Purchase Count: %{value}<br>"                     # always show count
        "Parent: %{parent}<br>"                            # parent id 
        "Season: %{customdata[0]}<br>"                     # season string 
        "Item: %{customdata[1]}<br>"                       # item string 
        "Purchase Amount (USD): %{customdata[2]:,.0f}<br>" 
        "Avg. Age: %{customdata[3]:.1f}<extra></extra>"
    )
)

fig.update_layout(margin=dict(t=10, l=10, r=10, b=10))
fig.show()


In [31]:

df_sum = df.groupby(["Age", "Gender"], as_index=False).agg({
    "Purchase Amount (USD)": "sum",
    "Previous Purchases": "sum"
})

fig = px.bar(
    df_sum,
    x="Age",
    y="Purchase Amount (USD)",
    color="Gender",
    color_discrete_sequence=pc.qualitative.Dark2,
    title="Total Purchase vs Previous Purchase by Age"
)

fig.add_trace(go.Scatter(
    x=df_sum["Age"],
    y=df_sum["Previous Purchases"],
    name="Previous Purchases",
    mode="lines+markers",
    line=dict(color="orange", width=3),
    yaxis="y2"  # pakai sumbu kanan
))

fig.update_layout(
    template="plotly_white",
    xaxis=dict(title="Age"),
    yaxis=dict(title="Total Purchase Amount (USD)"),
    yaxis2=dict(
        title="Previous Purchase (USD)",
        overlaying="y",
        side="right",
        showgrid=False
    ),
    legend=dict(
        title="Legend",
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="center",
        x=0.5
    )
)

fig.show()

In [32]:
df_rating = df["Review Rating"].value_counts().sort_index().reset_index()
df_rating.columns = ["Review Rating", "Count"]

avg_rating = df["Review Rating"].mean()

fig = px.bar(
    data_frame=df_rating,
    x="Review Rating",
    y="Count",
    # color="Review Rating",
    text="Count",
    color_discrete_sequence=pc.qualitative.Plotly
)

# tambahkan garis rata-rata (horizontal)
fig.add_shape(
    type="line",
    x0=avg_rating, x1=avg_rating,  # posisi garis pada rata-rata
    y0=0, y1=df_rating["Count"].max(),  # dari bawah ke atas
    line=dict(color="red", width=3, dash="dash"),
    name="Average Rating"
)

# tambahkan teks untuk menunjukkan nilai rata-rata
fig.add_annotation(
    x=avg_rating,
    y=df_rating["Count"].max(),
    text=f"Avg: {avg_rating:.2f}",
    showarrow=False,
    yshift=10,
    font=dict(color="red", size=12)
)

fig.update_traces(textposition="outside")

fig.update_layout(
    xaxis=dict(tickmode="linear"),
    template="plotly_white"
)

fig.show()

In [ ]:
df_shpping = df["Shipping Type"].value_counts().sort_index().reset_index()
df_shpping.columns = ["Shipping Type", "Count"]

fig_pie = px.pie(
    data_frame=df_shpping,
    names="Shipping Type",
    values="Count",
    title="Distribution of Shipping Type",
    hole=0.4,
    color_discrete_sequence=pc.sequential.Viridis 
)

fig_pie.update_traces(textinfo="percent+label")

fig_pie.update_layout(template="plotly_white")

fig_pie.show()

AttributeError: module 'plotly.colors' has no attribute 'sequantial'

In [36]:
df_payment = df["Payment Method"].value_counts().sort_index().reset_index()
df_payment.columns = ["Payment Method", "Count"]

fig_pie = px.pie(
    data_frame=df_payment,
    names="Payment Method",
    values="Count",
    title="Distribution of Payment Method",
    hole=0.4,
    color_discrete_sequence=pc.sequantial.Viridis
)

fig_pie.update_traces(textinfo="percent+label")

fig_pie.update_layout(template="plotly_white")

fig_pie.show()

AttributeError: module 'plotly.colors' has no attribute 'sequantial'